In [14]:
import pandas as pd
import locale
from holidays import country_holidays
import matplotlib.pyplot as plt
import seaborn as sns
import requests
#from tmdbv3api import TMDb, Movie
import os
from dotenv import load_dotenv
from tqdm.auto import tqdm
load_dotenv()

True

In [15]:
# Stile dei grafici
sns.set(style="whitegrid")

# Mappa dei giorni in italiano (corretta e sicura)
giorni_settimana = {
    "Monday": "lunedì",
    "Tuesday": "martedì",
    "Wednesday": "mercoledì",
    "Thursday": "giovedì",
    "Friday": "venerdì",
    "Saturday": "sabato",
    "Sunday": "domenica"
}

# Percorso assoluto basato sulla posizione del file corrente
# Definisci la variabile 'path' con il percorso corretto del file
path = os.path.join('input', 'dati_cinema_originali.csv')
file_path = path

In [16]:
# Caricamento con encoding
df = pd.read_csv(path, encoding="latin1", sep=";", dayfirst=True, on_bad_lines="skip")
print(df.head())

# Conversione della data
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y', errors='coerce')

# Combina data + ora reale
df['datetime'] = pd.to_datetime(df['date'].dt.date.astype(str) + ' ' + df['time'], errors='coerce')

# Ordina per data
df = df.sort_values("datetime").reset_index(drop=True)


# Estrazione e conversione
df["giorno_settimana_en"] = df["datetime"].dt.day_name()
df["giorno_settimana"] = df["giorno_settimana_en"].map(giorni_settimana)


# Pulizia colonna intermedia
df.drop(columns=["giorno_settimana_en"], inplace=True)


# Mese, anno, ora
df["mese"] = df["datetime"].dt.month
df["anno"] = df["datetime"].dt.year
df["ora"] = df["datetime"].dt.hour


# Fascia oraria
def get_fascia_oraria(ora):
    if ora == 10:
        return "mattina"
    elif 15 <= ora <= 17:
        return "pomeriggio"
    elif 20 <= ora <= 22:
        return "sera"
    else:
        return "altro"  # in caso ci fossero altri orari imprevisti

df["fascia_oraria"] = df["ora"].apply(get_fascia_oraria)


# Funzione per classificare le stagioni
def get_stagione(mese):
    if mese in [12, 1, 2]:
        return "inverno"
    elif mese in [3, 4, 5]:
        return "primavera"
    elif mese in [6, 7, 8]:
        return "estate"
    else:
        return "autunno"

df["stagione"] = df["mese"].apply(get_stagione)

# Weekend: True se sabato o domenica
df["weekend"] = df["giorno_settimana"].isin(["venerdì", "sabato", "domenica"])

# Anni unici presenti nel dataset
anni_presenti = df["anno"].unique()

# Festività italiane ufficiali per quegli anni
festivita_italiane = country_holidays("IT", years=anni_presenti)

# Colonna con nome della festività (NaN se non è una festività)
df["festività"] = df["datetime"].dt.date.map(festivita_italiane)
# Sostituisce i valori NaN con 'Nessuna'
df["festività"] = df["festività"].fillna("Nessuna")


df['total'] = df[['full_price', 'reduced', 'free']].sum(axis=1)
df['full_price'] = df['full_price'].fillna(0)
df['reduced'] = df['reduced'].fillna(0)
df['free'] = df['free'].fillna(0)

df.sample(5)

         date   time            title  full_price  reduced  free  total
0  14/09/2019  20:30      Il Re Leone        83.0    101.0   0.0    184
1  15/09/2019  17:00      Il Re Leone        53.0     67.0   1.0    121
2  15/09/2019  20:30      Il Re Leone        35.0     22.0   0.0     57
3  22/09/2019  17:00      Il Re Leone       114.0    127.0  13.0    254
4  28/09/2019  20:30  IT - Capitolo 2        75.0    123.0   0.0    198


,date,time,title,full_price,reduced,free,total,datetime,giorno_settimana,mese,anno,ora,fascia_oraria,stagione,weekend,festività
397,2024-02-09,17:00,Il Fantasma di Canterville,9.0,24.0,2.0,35.0,2024-02-09 17:00:00,venerdì,2,2024,17,pomeriggio,inverno,True,Nessuna
432,2024-03-31,20:30,Race for Glory,27.0,11.0,1.0,39.0,2024-03-31 20:30:00,domenica,3,2024,20,sera,primavera,True,Pasqua di Resurrezione
275,2023-04-08,20:30,John Wick 4,16.0,27.0,1.0,44.0,2023-04-08 20:30:00,sabato,4,2023,20,sera,primavera,True,Nessuna
445,2024-04-21,20:30,un mondo a parte,35.0,25.0,0.0,60.0,2024-04-21 20:30:00,domenica,4,2024,20,sera,primavera,True,Nessuna
597,2025-05-01,20:30,THUNDERBOLTS,3.0,18.0,0.0,21.0,2025-05-01 20:30:00,giovedì,5,2025,20,sera,primavera,False,Festa dei Lavoratori


In [17]:
# abilita progress_apply
tqdm.pandas(desc="Fetching weather")

def get_meteo(date, lat=46.0524, lon=11.45):
    url = (
        f"https://archive-api.open-meteo.com/v1/archive?"
        f"latitude={lat}&longitude={lon}&start_date={date}&end_date={date}"
        f"&daily=temperature_2m_max,temperature_2m_min,precipitation_sum"
        f"&timezone=Europe%2FRome"
    )
    resp = requests.get(url).json()
    if "daily" in resp:
        return {
            "temp_max": resp["daily"]["temperature_2m_max"][0],
            "temp_min": resp["daily"]["temperature_2m_min"][0],
            "precip_mm": resp["daily"]["precipitation_sum"][0]
        }
    else:
        return {"temp_max": None, "temp_min": None, "precip_mm": None}

# prepara le date
df["date_str"] = df["datetime"].dt.date.astype(str)

# usa progress_apply per la barra
meteo = df["date_str"].progress_apply(get_meteo)
meteo_df = pd.DataFrame(meteo.tolist())

# unisci
df = pd.concat([df, meteo_df], axis=1)


Fetching weather:   5%|▍         | 29/604 [00:44<14:42,  1.53s/it]


ReadTimeout: HTTPSConnectionPool(host='archive-api.open-meteo.com', port=443): Read timed out. (read timeout=None)